<a href="https://colab.research.google.com/github/irfanajr17/BelajarDE/blob/main/Homework_Spark2_Irfan_Alfath_ramadhan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [17]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [18]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.functions import lit
from pyspark.sql.types import *
from functools import reduce
from pyspark.sql import DataFrame

In [6]:
from google.colab import files
uploaded = files.upload()

Saving Dataset.zip to Dataset.zip


In [7]:
!unzip Dataset.zip

Archive:  Dataset.zip
   creating: Dataset/
 extracting: Dataset/cities1.csv     
  inflating: Dataset/cities1_schema.csv  
 extracting: Dataset/cities2.csv     
   creating: Dataset/exercise/
  inflating: Dataset/exercise/AkunTwitter_POS.csv  
  inflating: Dataset/exercise/HashtagTwitter_POS.csv  
  inflating: Dataset/exercise/Instagram_POS.json  
   creating: Dataset/json/
 extracting: Dataset/json/user1.json  
 extracting: Dataset/json/user2.json  
 extracting: Dataset/pcodes.csv      
  inflating: Dataset/people-no-pcode.csv  
  inflating: Dataset/purplecow.txt   
 extracting: Dataset/zcodes.csv      


In [19]:
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("Homework_Spark2_Irfan_Alfath_ramadhan") \
    .getOrCreate()

In [ ]:
spark

In [ ]:
AkunTwitter_POS = spark.read. \
  format("csv"). \
  option("inferSchema","true"). \
  option("header","true"). \
  load("Dataset/exercise/AkunTwitter_POS.csv")

AkunTwitter_POS.show()
AkunTwitter_POS.printSchema()

HashtagTwitter = spark.read. \
  format("csv"). \
  option("inferSchema","true"). \
  option("header","true"). \
  load("Dataset/exercise/HashtagTwitter_POS.csv")

HashtagTwitter.show()
HashtagTwitter.printSchema()

In [51]:
insta_df = spark.read.format("json").load("Dataset/exercise/Instagram_POS.json")
insta_df.show()
def FlatDF(schema, prefix=None):
        fields = []
        for field in schema.fields:
            name = prefix + '.' + field.name if prefix else field.name
            dtype = field.dataType
            if isinstance(dtype, ArrayType):
                dtype = dtype.elementType

            if isinstance(dtype, StructType):
                fields += FlatDF(dtype, prefix=name)
            else:
                fields.append(name)

        return fields
insta_df.schema
insta_df = insta_df.select(FlatDF(insta_df.schema))
insta_df.show()
insta_df.printSchema()
insta_df.show()
insta_df = insta_df.select(col("author").alias("Username"),col("comment").alias("Content"), col("key").alias("Source"))
insta_df.show()
insta_df.printSchema()
insta_final = insta_df.withColumn("Username", explode("Username")).withColumn("Content", explode("content")).withColumn("Username", col("Username")).withColumn("Content", col("Content"))
display(insta_final)


+--------------------+--------------------+--------------------+--------------------+--------------------+
|             caption|            comments|            datetime|            img_urls|                 key|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|Oranger menjaga s...|[[mamah.asyraf, C...|2019-07-13T04:06:...|[https://scontent...|https://www.insta...|
|Oranger, layanan ...|[[rizqimuhammad77...|2019-07-12T09:55:...|[https://scontent...|https://www.insta...|
|Selamat hari kope...|[[evan_mardiyanto...|2019-07-12T02:21:...|[https://scontent...|https://www.insta...|
|Kuliah? di @polte...|[[dicky.ra_, @pos...|2019-07-11T05:19:...|[https://scontent...|https://www.insta...|
|Selamat ulang tah...|[[hey.rudd, Kak.....|2019-07-11T01:22:...|[https://scontent...|https://www.insta...|
|Kiriman lewat Pos...|[[rienlilitanty, ...|2019-07-10T10:46:...|[https://scontent...|https://www.insta...|
|Sedang mencari ka...|[[nurdianing, m

DataFrame[Username: string, Content: string, Source: string]

In [58]:
AP = AkunTwitter_POS.select(AkunTwitter_POS["username"], AkunTwitter_POS["tweet"].alias("content"),lit("Twitter").alias("Source")).where(AkunTwitter_POS["username"] == 'posindonesia').show()
HT = HashtagTwitter.select(HashtagTwitter["username"],HashtagTwitter["tweet"].alias("content"),lit("Twitter").alias("Source")).where(HashtagTwitter["username"] == 'posindonesia').show()
IP = insta_final.select(col("Username").alias("Username"), col("Content").alias("Content"), lit("Instagram").alias("Source")).where(insta_final["Username"]=="posindonesia.ig").show()


+------------+--------------------+-------+
|    username|             content| Source|
+------------+--------------------+-------+
|posindonesia|Baik sahabat, tel...|Twitter|
|posindonesia|Hai sahabat. Kiri...|Twitter|
|posindonesia|/ handphone pener...|Twitter|
|posindonesia|Hai sahabat, kiri...|Twitter|
|posindonesia|Oranger menjaga s...|Twitter|
|posindonesia|Hai sahabat. Untu...|Twitter|
|posindonesia|Terima kasih tela...|Twitter|
|posindonesia|Hai sahabat. Moho...|Twitter|
|posindonesia|Hai sahabat. Moho...|Twitter|
|posindonesia|Sama - sama sahab...|Twitter|
|posindonesia|Hai sahabat. Kiri...|Twitter|
|posindonesia|biaya pelalubeaan...|Twitter|
|posindonesia|Halo Sahabat, seb...|Twitter|
|posindonesia|Berdasarkan hasil...|Twitter|
|posindonesia|Mohon diinformasi...|Twitter|
|posindonesia|kami informasikan...|Twitter|
|posindonesia|Baik Sahabat, Moh...|Twitter|
|posindonesia|Halo Sahabat. Moh...|Twitter|
|posindonesia|Halo Sahabat pos....|Twitter|
|posindonesia|Baik sahabat moho.